In [2]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [40]:
df = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [41]:
df = df[(df['press'] == 'TechSpot') | (df['press'] == 'MakeUseOf')]
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
715,"{'id': None, 'name': 'TechSpot'}",Steven Walton,In Hindsight: Some of the Worst CPU/GPUs Purch...,Today we&#39;re discussing what we feel were t...,https://www.techspot.com/article/1523-worst-cp...,https://static.techspot.com/articles-info/1523...,2017-11-13T07:00:00Z,If you want to learn about what graphics card ...,TechSpot
2354,"{'id': None, 'name': 'TechSpot'}",Rob Thubron,Coinbase goes after institutional investors wi...,Bitcoin&rsquo;s value may have skyrocketed rec...,https://www.techspot.com/news/71930-coinbase-g...,https://static.techspot.com/images2/news/ts3_t...,2017-11-17T12:32:00Z,Bitcoin’s value may have skyrocketed recently—...,TechSpot
2475,"{'id': None, 'name': 'TechSpot'}",Shawn Knight,Survey of Bitcoin owners reveals that most are...,With Bitcoin values continuing to climb (they’...,https://www.techspot.com/news/71937-survey-bit...,https://static.techspot.com/images2/news/ts3_t...,2017-11-17T17:54:00Z,With Bitcoin values continuing to climb (they’...,TechSpot
3515,"{'id': None, 'name': 'TechSpot'}",Shawn Knight,Cryptocurrency start-up vanishes after raising...,Cryptocurrency enthusiasts across the web are ...,https://www.techspot.com/news/71983-cryptocurr...,https://static.techspot.com/images2/news/ts3_t...,2017-11-21T18:53:00Z,Cryptocurrency enthusiasts across the web are ...,TechSpot
3688,"{'id': None, 'name': 'TechSpot'}",Rob Thubron,11 Tech Products Were Supposed to Fail... But ...,You&#39;ll often find that when a new product ...,https://www.techspot.com/article/1522-tech-pro...,https://static.techspot.com/articles-info/1522...,2017-11-22T05:00:00Z,You'll often find that when a new product or s...,TechSpot
...,...,...,...,...,...,...,...,...,...
740214,"{'id': None, 'name': 'MakeUseOf'}",Katie Rees,How to Transfer Your Crypto to a Cold Storage ...,Don't take risks with your crypto. Use a cold ...,https://www.makeuseof.com/how-transfer-crypto-...,https://static1.makeuseofimages.com/wordpress/...,2023-09-21T15:30:26Z,Having a software crypto wallet is all fun and...,MakeUseOf
740622,"{'id': None, 'name': 'MakeUseOf'}",Shubham Pandey,Bitcoin Crash vs. Correction: What's the Diffe...,Understanding the difference can help you navi...,https://www.makeuseof.com/bitcoin-crash-vs-cor...,https://static1.makeuseofimages.com/wordpress/...,2023-09-22T11:15:25Z,Key Takeaways\r\n<ul><li> A crypto crash is a ...,MakeUseOf
740862,"{'id': None, 'name': 'TechSpot'}",Cal Jeffrey,"System admin and wife plead guilty to ""massive...","Brad and Dusti Pearce of Tuttle, Oklahoma, con...",https://www.techspot.com/news/100257-system-ad...,https://www.techspot.com/images2/news/bigimage...,2023-09-22T20:03:00Z,In a nutshell: A system administrator and his ...,TechSpot
741794,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,"What Are MetaMask Snaps, and How Do They Work?",MetaMask Snaps will revolutionize cryptocurren...,https://www.makeuseof.com/what-are-metamask-sn...,https://static1.makeuseofimages.com/wordpress/...,2023-09-25T21:00:30Z,"On September 12, 2023, ConsenSys Software Inc....",MakeUseOf


In [42]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(df):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        url = row['url']
        press = row['press']
        
        # press 값에 따른 selector 설정
        if press == 'TechSpot':
            selector = '.wrapperMobileOnly'
        elif press == 'MakeUseOf':
            selector = '.content-block-regular'
        # elif press == 'Al Jazeera English':
        #     selector = '.css-ibbk12'

        else:
            print(f"Unknown press value: {press} for URL: {url}")
            contents.append('')
            continue

        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            article_elements = soup.select(selector)

            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents


In [43]:
head = df.head()
head['content_full'] = get_contents(head)
head

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:05<00:00,  1.17s/it]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_21300\2752012077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
715,"{'id': None, 'name': 'TechSpot'}",Steven Walton,In Hindsight: Some of the Worst CPU/GPUs Purch...,Today we&#39;re discussing what we feel were t...,https://www.techspot.com/article/1523-worst-cp...,https://static.techspot.com/articles-info/1523...,2017-11-13T07:00:00Z,If you want to learn about what graphics card ...,TechSpot,\n\n\n\n\n\nFacebook\n\n\nTwitter\n\n\nReddit\...
2354,"{'id': None, 'name': 'TechSpot'}",Rob Thubron,Coinbase goes after institutional investors wi...,Bitcoin&rsquo;s value may have skyrocketed rec...,https://www.techspot.com/news/71930-coinbase-g...,https://static.techspot.com/images2/news/ts3_t...,2017-11-17T12:32:00Z,Bitcoin’s value may have skyrocketed recently—...,TechSpot,\nTechSpot is celebrating its 25th anniversary...
2475,"{'id': None, 'name': 'TechSpot'}",Shawn Knight,Survey of Bitcoin owners reveals that most are...,With Bitcoin values continuing to climb (they’...,https://www.techspot.com/news/71937-survey-bit...,https://static.techspot.com/images2/news/ts3_t...,2017-11-17T17:54:00Z,With Bitcoin values continuing to climb (they’...,TechSpot,\nTechSpot is celebrating its 25th anniversary...
3515,"{'id': None, 'name': 'TechSpot'}",Shawn Knight,Cryptocurrency start-up vanishes after raising...,Cryptocurrency enthusiasts across the web are ...,https://www.techspot.com/news/71983-cryptocurr...,https://static.techspot.com/images2/news/ts3_t...,2017-11-21T18:53:00Z,Cryptocurrency enthusiasts across the web are ...,TechSpot,\nTechSpot is celebrating its 25th anniversary...
3688,"{'id': None, 'name': 'TechSpot'}",Rob Thubron,11 Tech Products Were Supposed to Fail... But ...,You&#39;ll often find that when a new product ...,https://www.techspot.com/article/1522-tech-pro...,https://static.techspot.com/articles-info/1522...,2017-11-22T05:00:00Z,You'll often find that when a new product or s...,TechSpot,\n\n\n\nIndex\n\n\nArticle Index\n\nIntro\nAma...


In [44]:
# Adding the content to the DataFrame
df['content_full'] = get_contents(df)
df

100%|██████████| 1363/1363 [31:24<00:00,  1.38s/it]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_21300\4181533438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content_full'] = get_contents(df)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
715,"{'id': None, 'name': 'TechSpot'}",Steven Walton,In Hindsight: Some of the Worst CPU/GPUs Purch...,Today we&#39;re discussing what we feel were t...,https://www.techspot.com/article/1523-worst-cp...,https://static.techspot.com/articles-info/1523...,2017-11-13T07:00:00Z,If you want to learn about what graphics card ...,TechSpot,\n\n\n\n\n\nFacebook\n\n\nTwitter\n\n\nReddit\...
2354,"{'id': None, 'name': 'TechSpot'}",Rob Thubron,Coinbase goes after institutional investors wi...,Bitcoin&rsquo;s value may have skyrocketed rec...,https://www.techspot.com/news/71930-coinbase-g...,https://static.techspot.com/images2/news/ts3_t...,2017-11-17T12:32:00Z,Bitcoin’s value may have skyrocketed recently—...,TechSpot,\nTechSpot is celebrating its 25th anniversary...
2475,"{'id': None, 'name': 'TechSpot'}",Shawn Knight,Survey of Bitcoin owners reveals that most are...,With Bitcoin values continuing to climb (they’...,https://www.techspot.com/news/71937-survey-bit...,https://static.techspot.com/images2/news/ts3_t...,2017-11-17T17:54:00Z,With Bitcoin values continuing to climb (they’...,TechSpot,\nTechSpot is celebrating its 25th anniversary...
3515,"{'id': None, 'name': 'TechSpot'}",Shawn Knight,Cryptocurrency start-up vanishes after raising...,Cryptocurrency enthusiasts across the web are ...,https://www.techspot.com/news/71983-cryptocurr...,https://static.techspot.com/images2/news/ts3_t...,2017-11-21T18:53:00Z,Cryptocurrency enthusiasts across the web are ...,TechSpot,\nTechSpot is celebrating its 25th anniversary...
3688,"{'id': None, 'name': 'TechSpot'}",Rob Thubron,11 Tech Products Were Supposed to Fail... But ...,You&#39;ll often find that when a new product ...,https://www.techspot.com/article/1522-tech-pro...,https://static.techspot.com/articles-info/1522...,2017-11-22T05:00:00Z,You'll often find that when a new product or s...,TechSpot,\n\n\n\nIndex\n\n\nArticle Index\n\nIntro\nAma...
...,...,...,...,...,...,...,...,...,...,...
740214,"{'id': None, 'name': 'MakeUseOf'}",Katie Rees,How to Transfer Your Crypto to a Cold Storage ...,Don't take risks with your crypto. Use a cold ...,https://www.makeuseof.com/how-transfer-crypto-...,https://static1.makeuseofimages.com/wordpress/...,2023-09-21T15:30:26Z,Having a software crypto wallet is all fun and...,MakeUseOf,\nHaving a software crypto wallet is all fun a...
740622,"{'id': None, 'name': 'MakeUseOf'}",Shubham Pandey,Bitcoin Crash vs. Correction: What's the Diffe...,Understanding the difference can help you navi...,https://www.makeuseof.com/bitcoin-crash-vs-cor...,https://static1.makeuseofimages.com/wordpress/...,2023-09-22T11:15:25Z,Key Takeaways\r\n<ul><li> A crypto crash is a ...,MakeUseOf,\n\nKey Takeaways\n\n A crypto crash is a sud...
740862,"{'id': None, 'name': 'TechSpot'}",Cal Jeffrey,"System admin and wife plead guilty to ""massive...","Brad and Dusti Pearce of Tuttle, Oklahoma, con...",https://www.techspot.com/news/100257-system-ad...,https://www.techspot.com/images2/news/bigimage...,2023-09-22T20:03:00Z,In a nutshell: A system administrator and his ...,TechSpot,\nTechSpot is celebrating its 25th anniversary...
741794,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,"What Are MetaMask Snaps, and How Do They Work?",MetaMask Snaps will revolutionize cryptocurren...,https://www.makeuseof.com/what-are-metamask-sn...,https://static1.makeuseofimages.com/wordpress/...,2023-09-25T21:00:30Z,"On September 12, 2023, ConsenSys Software Inc....",MakeUseOf,"\nOn September 12, 2023, ConsenSys Software In..."


In [285]:
head['url'].iloc[3]

'http://www.thestar.com.my/business/business-news/2017/11/13/bitcoin-tumble-erases-us$38b-as-rival-cryptocurrency-gains/'

In [296]:
head['content_full'].iloc[0]

"\n\n\n\n\n\nLipin Pappachan, Tech Times\n                    \n                21 November 2017, 09:11 am\n\n\n\nNearly $31 million worth of bitcoin currency was wiped off from cryptocurrency\xa0startup Tethers' Treasury account. The startup is in business with bitcoin exchanges and the hacker was able to move the bitcoin to an unauthorized account.\nTether has released the details of the attack and precautionary measures on its website for its users.\nDetails Of The Attack That Stole $31 Million\nOn Nov. 19, 2017, the Treasury account of Tether was compromised by a hacker. The bitcoins were maliciously transferred from Tether to the following address: 16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r by the hacker. Although the fund address has now been flagged, the curtailment of further damage will depend on how the startup is able to control the spread of this fund in the broader ecosystem. Tether has requested people who trade in bitcoins to not to purchase USDT tokens from the above address.\n

In [45]:
empty_rows = df[df['content_full'] == '']
print(len(empty_rows))

2


In [22]:
empty_rows['url'].iloc[40]

'https://boingboing.net/2019/05/16/atlas-launched.html'

In [48]:
MakeUseOf = df[df['press'] == 'MakeUseOf']
MakeUseOf

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
226103,"{'id': None, 'name': 'MakeUseOf'}",Sandy Writtenhouse,"How to Set Up Google Alerts for News, Shopping...","When you want to find something on the web, yo...",https://www.makeuseof.com/tag/how-to-set-up-go...,https://static.makeuseof.com/wp-content/upload...,2019-12-11T18:00:51Z,"When you want to find something on the web, yo...",MakeUseOf,"\nWhen you want to find something on the web, ..."
227282,"{'id': None, 'name': 'MakeUseOf'}",Georgina Torbet,How Much Energy Does Your PC Use? (And 8 Ways ...,As people become more aware of the impact of t...,https://www.makeuseof.com/tag/much-energy-pc-u...,https://static.makeuseof.com/wp-content/upload...,2019-12-18T15:00:00Z,As people become more aware of the impact of t...,MakeUseOf,\nAs people become more aware of the impact of...
236393,"{'id': None, 'name': 'MakeUseOf'}",Georgina Torbet,5 Security Reasons Not to Download Cracked Sof...,Purchasing software can be expensive. When you...,https://www.makeuseof.com/tag/security-reasons...,https://static.makeuseof.com/wp-content/upload...,2020-02-18T12:00:00Z,Purchasing software can be expensive. When you...,MakeUseOf,\nPurchasing software can be expensive. When y...
236541,"{'id': None, 'name': 'MakeUseOf'}",Christian Cawley,"What Is Bitcoin, How Is it Worth So Much, and ...",This week’s Really Useful Podcast takes a look...,https://www.makeuseof.com/tag/what-is-bitcoin-...,https://static.makeuseof.com/wp-content/upload...,2020-02-19T04:30:18Z,This week’s Really Useful Podcast takes a look...,MakeUseOf,\nThis week's Really Useful Podcast takes a lo...
237356,"{'id': None, 'name': 'MakeUseOf'}",Christian Cawley,The 9 Best Browsers for Linux,It’s virtually impossible to use a modern comp...,https://www.makeuseof.com/tag/best-linux-brows...,https://static.makeuseof.com/wp-content/upload...,2020-02-24T12:00:00Z,It’s virtually impossible to use a modern comp...,MakeUseOf,\nIt's virtually impossible to use a modern co...
...,...,...,...,...,...,...,...,...,...,...
740106,"{'id': None, 'name': 'MakeUseOf'}",Elvis Gwaro,Top 10 Blockchain Domain Name Systems,"Web3 is picking up pace, and grabbing a blockc...",https://www.makeuseof.com/top-blockchain-domai...,https://static1.makeuseofimages.com/wordpress/...,2023-09-21T12:45:25Z,Key Takeaways\r\n<ul><li> Blockchain domains l...,MakeUseOf,\n\nKey Takeaways\n\n Blockchain domains like...
740214,"{'id': None, 'name': 'MakeUseOf'}",Katie Rees,How to Transfer Your Crypto to a Cold Storage ...,Don't take risks with your crypto. Use a cold ...,https://www.makeuseof.com/how-transfer-crypto-...,https://static1.makeuseofimages.com/wordpress/...,2023-09-21T15:30:26Z,Having a software crypto wallet is all fun and...,MakeUseOf,\nHaving a software crypto wallet is all fun a...
740622,"{'id': None, 'name': 'MakeUseOf'}",Shubham Pandey,Bitcoin Crash vs. Correction: What's the Diffe...,Understanding the difference can help you navi...,https://www.makeuseof.com/bitcoin-crash-vs-cor...,https://static1.makeuseofimages.com/wordpress/...,2023-09-22T11:15:25Z,Key Takeaways\r\n<ul><li> A crypto crash is a ...,MakeUseOf,\n\nKey Takeaways\n\n A crypto crash is a sud...
741794,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,"What Are MetaMask Snaps, and How Do They Work?",MetaMask Snaps will revolutionize cryptocurren...,https://www.makeuseof.com/what-are-metamask-sn...,https://static1.makeuseofimages.com/wordpress/...,2023-09-25T21:00:30Z,"On September 12, 2023, ConsenSys Software Inc....",MakeUseOf,"\nOn September 12, 2023, ConsenSys Software In..."


<!-- # empty_rows['url'].iloc[0] -->

In [49]:
MakeUseOf.to_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\MakeUseOf_149.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content